In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

## 1. Data

In [3]:
# Import the original dataset

df = pd.read_csv("data/sbl_grouped_cmpls.csv").fillna("")

In [4]:
len(df)

659

In [4]:
pd.set_option("display.max_columns", None)

In [5]:
df[(df.construction=="non-prep") & (df.cmpl_anim =="anim")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse,construction
33,216346,<WP[,MT,Isaiah,11,14,<PW,W <PW B KTP PLCTJM JMH JXDW JBZW >T BNJ QDM >D...,,qal,perf,W <PW B KTP PLCTJM JMH,,PLCTJM,PLCTJ/,towards the Philistines,0.0,vc,gens,anim,und,nmpr,simple,factive,,,,,,,non-prep


## 2. Chi Square Test of Independence: Cmpl Construction and Cmpl Animacy

In [11]:
# Contingency table
contingency_table_anim = pd.crosstab(df['construction'], df['cmpl_anim'])

# Convert the contingency table to a numpy array
observed = contingency_table_anim.values

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(observed)

# Output the results
print("Chi-Square Statistic:", chi2)
print("P-value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies: the expected frequencies should all be >= 5 for the Chi2 test to be appropriate\n", expected)



Chi-Square Statistic: 58.96218036195984
P-value: 1.6073179256695774e-14
Degrees of Freedom: 1
Expected Frequencies: the expected frequencies should all be >= 5 for the Chi2 test to be appropriate
 [[ 36.53869499  84.46130501]
 [162.46130501 375.53869499]]


In [12]:
contingency_table_anim

cmpl_anim,anim,inanim
construction,,
non-prep,1,120
prep,198,340


## 3. Chi Square Test of Independence: Cmpl Construction and Cmpl Definiteness

In [13]:
# Contingency table
contingency_table_det = pd.crosstab(df['construction'], df['cmpl_det'])

# Convert the contingency table to a numpy array
observed = contingency_table_det.values

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(observed)

# Output the results
print("Chi-Square Statistic:", chi2)
print("P-value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies: the expected frequencies should all be >= 5 for the Chi2 test to be appropriate\n", expected)

Chi-Square Statistic: 2.624187150649214
P-value: 0.10524638986744972
Degrees of Freedom: 1
Expected Frequencies: the expected frequencies should all be >= 5 for the Chi2 test to be appropriate
 [[114.20637329   6.79362671]
 [507.79362671  30.20637329]]


In [14]:
contingency_table_det

cmpl_det,det,und
construction,,
non-prep,110,11
prep,512,26
